In [1]:
!python3 -V

Python 3.13.3


In [1]:
import pandas as pd
import pickle

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [29]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')
df
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)


In [30]:
#Q1 - Number of columns in january data
len(df.columns)

20

In [31]:
# Q2 - standard deviation of trips duration in January
df['duration'].std()

np.float64(42.59435124195458)

In [32]:
r_before = len(df)

In [33]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [37]:
#Q3 - fraction of reacords left after dropping outliers
r_after = len(df)
r_after/r_before

0.9812202822125979

In [42]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [43]:
df_train = read_dataframe('../data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('../data/yellow_tripdata_2023-02.parquet')

In [50]:
len(df_train), len(df_val)

(3009173, 2855951)

In [52]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [57]:
# categorical = ['PU_DO']
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [60]:
# Q4 - dimensionality of feature matrix
X_train.shape

(3009173, 516)

In [66]:
# categorical = ['PU_DO']
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [67]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [68]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [69]:
# Q5 - RMSE of train
y_pred = lr.predict(X_train)
root_mean_squared_error(y_train, y_pred)

7.658403512356688

In [71]:
# Q6 - RMSE of val
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.819945413902019